In [26]:
from bs4 import BeautifulSoup 
from selenium import webdriver
# The following packages will also be used in this tutorial
import pandas as pd # All database operations
import numpy as np  # Numerical operations
import time         # Tracking time
import requests     # HTTP requests
import re           # String manipulation
from sklearn.feature_extraction.text import CountVectorizer # BagOfWords (cleaning)
from joblib import Parallel, delayed # Parallellization of tasks

In [27]:
def getPage(url):
	''' returns a soup object that contains all the information 
	of a certain webpage'''
	result = requests.get(url)
	content = result.content
	return BeautifulSoup(content, features = "lxml")

url_page ="https://www.airbnb.fr/s/Paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2020-12-07&checkout=2020-12-08&source=structured_search_input_header&search_type=filter_change&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ"
page=getPage(url_page)

In [28]:
def getRoomClasses(soupPage):
	''' This function returns all the listings that can 
	be found on the page in a list.'''
	rooms = soupPage.findAll("div", {"class": "_8ssblpx"})
	result = []
	for room in rooms:
		result.append(room)
	return result
listing=getRoomClasses(page)

In [29]:
##commentaires
def getNbCommentaire(listing):
	''' Returns the guest information'''
	try:
		return listing.find("span", {"class":"_a7a5sx"}).text 
	except:
		return "pas de commentaire"

In [30]:
##evaluation
def getEvaluation(listing):
	''' Returns the guest information'''
	try:
		return listing.find("span", {"class":"_10fy1f8"}).text 
	except:
		return "pas d'évaluation"


In [31]:
## lien du logement
def getListingLink(listing):
	''' Returns the guest information'''
	try:
		return "http://airbnb.com" + listing.find("a")["href"]
	except:
		return "pas de lien"

In [32]:
## titre du logement
def getListingTitle(listing):
	''' Returns the guest information'''
	try:
		return listing.find("meta")["content"]
	except:
		return "pas de titre du logement"

In [33]:
##description du logement
def getTopRow(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class": "_167qordg"}).text
	except:
		return "pas de description du logement"

In [34]:
##information sur le nombre de chambre/douche
def getRoomInfo(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class":"_kqh46o"}).text
	except:
		return "pas d'infos sur chambre/douche"

In [35]:
##prix
def getPrix(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class":"_1fwiw8gv"}).text
	except:
		return "pas d'infos sur le prix"

In [36]:

getNbCommentaire(listing[5])
getPrix(listing[0])
getEvaluation(listing[5])
getListingTitle(listing[6])

'Paris Canal Saint Martin appart hôtel - null - Paris'

In [37]:
## extraction des infos de tous les logements d'une page

def extractinfo(page):
    df = pd.DataFrame(columns = ['title', 'link',"nbComments","prix","topRow","evaluation","roomInfo"]) ##,"evaluation", ,'nbComments'
    new=[]
    listing=getRoomClasses(page)
    for i in range(1, len(listing)):
        new.append(getListingTitle(listing[i]))
        new.append(getListingLink(listing[i]))
        new.append(getNbCommentaire(listing[i]))
        new.append(getPrix(listing[i]))
        new.append(getTopRow(listing[i]))
        new.append(getEvaluation(listing[i]))
        new.append(getRoomInfo(listing[i]))
        df.loc[i]=new
        new=[]
    return df

In [38]:
extractinfo(page)

,title,link,nbComments,prix,topRow,evaluation,roomInfo
1,Sama Paris - Studio le plus élevé de France -...,http://airbnb.com/rooms/32628714?check_in=2020...,(120),Prix initial :164€Prix réduit :125€ / nuit,Chambre privée à Puteaux,4.75,"4 voyageurs · 1 chambre · 2 lits · 1,5 salle d..."
2,Saint Honoré - Vendome - null - Paris,http://airbnb.com/rooms/38446941?check_in=2020...,(64),Prix initial :291€Prix réduit :171€ / nuit,Logement entier à 1er arrondissement,4.92,3 voyageurs · 1 chambre · 1 lit · 1 salle de bain
3,"Jolie chambre cosy avec un lit double, située ...",http://airbnb.com/rooms/26316846?check_in=2020...,(254),Prix :61€ / nuit,Chambre d'hôtel à 10e arrondissement,4.47,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
4,Design Appartement coeur de Paris - null - Par...,http://airbnb.com/rooms/28967540?check_in=2020...,(179),Prix initial :193€Prix réduit :137€ / nuit,Logement entier à 10e arrondissement,4.43,4 voyageurs · 1 chambre · 2 lits · 1 salle de ...
5,Studio Champs Elysées et Arc de Triomphe - nul...,http://airbnb.com/rooms/33969442?check_in=2020...,(70),Prix :36€ / nuit,Logement entier à Ternes,4.12,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
6,Paris Canal Saint Martin appart hôtel - null -...,http://airbnb.com/rooms/46000504?check_in=2020...,pas de commentaire,Prix :56€ / nuit,Logement entier dans une résidence de tourisme...,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
7,French Theory - La Chambre d'Etude - null - P...,http://airbnb.com/rooms/39410968?check_in=2020...,(121),Prix initial :78€Prix réduit :66€ / nuit,Chambre dans un boutique-hôtel à Quartier Latin,4.81,1 voyageur · 1 chambre · 1 lit · 1 salle de ba...
8,Studio au pied de la tour Eiffel - null - Paris,http://airbnb.com/rooms/46477198?check_in=2020...,pas de commentaire,Prix :59€ / nuit,Appartement en résidence entier à Tour Eiffel ...,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
9,Convivialité & PDJ offert en plein centre de P...,http://airbnb.com/rooms/27004700?check_in=2020...,(187),Prix :107€ / nuit,Chambre d'hôtel à Invalides - Ecole Militaire,4.86,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
10,Studio-Saint-Honoré-Vendôme-Louvre - null - Paris,http://airbnb.com/rooms/39197481?check_in=2020...,(51),Prix initial :163€Prix réduit :125€ / nuit,Logement entier à 1er arrondissement,4.94,3 voyageurs · Studio · 1 lit · 1 salle de bain


In [39]:
url_page

'https://www.airbnb.fr/s/Paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&checkin=2020-12-07&checkout=2020-12-08&source=structured_search_input_header&search_type=filter_change&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ'

In [40]:
## il faut ajouter des dates aux liens pour voir les prix
def findNextPage(soupPage):
	''' Finds the next page with listings if it exists '''
	try:
		nextpage = "https://airbnb.com" + soupPage.find("a", {"class": "_za9j7e"})["href"]
	except: # When he can't find the button, I assume he reached the end
		nextpage = "no next page"
	return nextpage
url=findNextPage(page)
url

'https://airbnb.com/s/Paris/homes?tab_id=home_tab&checkin=2020-12-07&refinement_paths%5B%5D=%2Fhomes&source=structured_search_input_header&checkout=2020-12-08&search_type=pagination&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=008040ac-c75c-4c14-85be-4bb5b829a130&items_offset=20&section_offset=3'

In [41]:
## nombre de page du site web
def getPages(url):
	''' This function returns all the links to the pages containing 
	listings for one particular city '''
	result = []
	while url != "no next page": 
		page = getPage(url)
		result = result + [page]
		url = findNextPage(page)
	return result
URL=getPages(url_page)
len(URL)

15

In [42]:
## Scrapper toutes les infos sur les logements
def extractPages(url):
	''' This function outputs a dataframe that contains all information of a particular
	city. It thus contains information of multiple listings coming from multiple pages.'''
	pages = getPages(url)
	# Do for the first element to initialize the dataframe
	df =extractinfo(pages[0])
	# Loop over all other elements of the dataframe
	for pagenumber in range(1, len(pages)):
		df = df.append(extractinfo(pages[pagenumber]))
	return df
extractPages(url_page)

,title,link,nbComments,prix,topRow,evaluation,roomInfo
1,Paris Canal Saint Martin appart hôtel - null -...,http://airbnb.com/rooms/46000504?check_in=2020...,pas de commentaire,Prix :56€ / nuit,Logement entier dans une résidence de tourisme...,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
2,Saint Honoré - Vendome - null - Paris,http://airbnb.com/rooms/38446941?check_in=2020...,(64),Prix initial :291€Prix réduit :171€ / nuit,Logement entier à 1er arrondissement,4.92,3 voyageurs · 1 chambre · 1 lit · 1 salle de bain
3,Design Appartement coeur de Paris - null - Par...,http://airbnb.com/rooms/28967540?check_in=2020...,(179),Prix initial :193€Prix réduit :137€ / nuit,Logement entier à 10e arrondissement,4.43,4 voyageurs · 1 chambre · 2 lits · 1 salle de ...
4,Superbe Studio 2P avec Jacuzzi - Opéra - 41 - ...,http://airbnb.com/rooms/46506547?check_in=2020...,pas de commentaire,Prix :145€ / nuit,Logement entier à 2e arrondissement,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
5,"Double/Twin Room, close to Opera and the Louvr...",http://airbnb.com/rooms/26244787?check_in=2020...,(98),Prix :83€ / nuit,Chambre d'hôtel à 2e arrondissement,4.88,2 voyageurs · 1 chambre · 1 salle de bain
...,...,...,...,...,...,...,...
15,Calme aux portes de Paris Maison examen 5MN - ...,http://airbnb.com/rooms/39149867?check_in=2020...,(29),Price:$79 / night,Entire apartment in Alésia,4.03,2 guests · 1 bedroom · 1 bed · 1 bath
16,"Rayz Eiffel, Studio Deluxe - null - Paris",http://airbnb.com/rooms/38776391?check_in=2020...,(5),Previous price:$202Discounted price:$158 / night,Room in serviced apartment in Tour Eiffel - Ch...,5.0,2 guests · 1 bedroom · 1 bed · 1 private bath
17,Cosy studio in the 20th arrondissement - null ...,http://airbnb.com/rooms/34490367?check_in=2020...,(6),Price:$65 / night,Entire apartment in Père Lachaise - Ménilmontant,4.50,2 guests · Studio · 1 bed · 1 bath
18,L&D 312- 1 bunk in room in paris 75019 - null ...,http://airbnb.com/rooms/46048568?check_in=2020...,pas de commentaire,Price:$28 / night,Shared room in La Villette,pas d'évaluation,6 guests · 1 bedroom · 6 beds · 1.5 shared baths


In [43]:
##base de tous les logements avec les évaluations
data=pd.DataFrame.from_dict(extractPages(url_page))

In [44]:
data.head()

,title,link,nbComments,prix,topRow,evaluation,roomInfo
1,Studio Champs Elysées et Arc de Triomphe - nul...,http://airbnb.com/rooms/33969442?check_in=2020...,(70),Prix :36€ / nuit,Logement entier à Ternes,4.12,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
2,Paris Canal Saint Martin appart hôtel - null -...,http://airbnb.com/rooms/46000504?check_in=2020...,pas de commentaire,Prix :56€ / nuit,Logement entier dans une résidence de tourisme...,pas d'évaluation,2 voyageurs · Studio · 1 lit · 1 salle de bain
3,Paris La Defense: Apt with Terrasse & Eiffel v...,http://airbnb.com/rooms/45572921?check_in=2020...,(12),Prix initial :166€Prix réduit :114€ / nuit,Logement entier à Courbevoie,4.42,"5 voyageurs · 2 chambres · 3 lits · 2,5 salles..."
4,"Jolie chambre cosy avec un lit double, située ...",http://airbnb.com/rooms/26316846?check_in=2020...,(254),Prix :61€ / nuit,Chambre d'hôtel à 10e arrondissement,4.47,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
5,Superbe loft en plein Paris - null - Paris,http://airbnb.com/rooms/30146057?check_in=2020...,(146),Prix initial :147€Prix réduit :129€ / nuit,Loft entier à 9e arrondissement,4.70,4 voyageurs · 2 chambres · 2 lits · 1 salle de...


In [54]:
v=data['link']

In [ ]:
## scrapper les commentaires de chaque logement

In [2]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains 


In [3]:
driver = webdriver.Chrome()

In [4]:
## installer chrome driver
driver_path = 'D:/chromedriver.exe'
driver = webdriver.Chrome(executable_path=driver_path)
#options = webdriver.ChromeOptions()
#options.binary_location = "D:/ChromeSetup.exe"
#driver = webdriver.Chrome(chrome_options=options, executable_path="D:/chromedriver.exe")

In [ ]:
#options = webdriver.ChromeOptions()
#options.binary_location = "D:/ChromeSetup.exe"
#driver = webdriver.Chrome(chrome_options=options, executable_path="D:/chromedriver.exe")

In [22]:
def setupDriver(url, waiting_time = 5):
	''' Initializes the driver of selenium'''
	driver = webdriver.Chrome()
	driver.get(url)
	time.sleep(waiting_time) 
	return driver

In [142]:
## charger chaque listing
def getJSpage(url):
	driver = setupDriver(url)
	html = driver.page_source
	driver.close()
	return BeautifulSoup(html, features="lxml")
js=getJSpage("https://www.airbnb.fr/rooms/27004700?check_in=2020-12-07&check_out=2020-12-08&source_impression_id=p3_1607170145_Q1fJuM0yFZY3ixvf")
# lien des commentaires de chaque listing
def getCommentLink(soupPage):
	try:
		return "https://www.airbnb.fr" + soupPage.find("div", {"class": "_19qg1ru"}).find("a")["href"]
	except:
		return "pas de lien"
url_comment=getCommentLink(js)

In [143]:
url_comment

'https://www.airbnb.fr/rooms/27004700/reviews?check_in=2020-12-07&check_out=2020-12-08&source_impression_id=p3_1607170145_Q1fJuM0yFZY3ixvf'

In [184]:
def getJSpageComments(url):
    driver=setupDriver(url)
    #driver.maximize_window()
    #time.sleep(10)
    html = driver.find_element_by_tag_name('html')
    html.send_keys(Keys.PAGE_DOWN)
    html=driver.page_source
    driver.close()
    return BeautifulSoup(html,features="lxml")

In [24]:
##Scrolling
driver=setupDriver("https://www.airbnb.fr/rooms/32628714/reviews?check_in=2020-12-07&amp;check_out=2020-12-08&amp;source_impression_id=p3_1607154423_w7emPzjZq6Y3FZp2")
num_current_comment_found = 0
pre_scroll_num_of_comment=-1
comment_class_when_scrolling  = '_1gjypya'
time.sleep(5)
while (num_current_comment_found != pre_scroll_num_of_comment) :           
            visible_comments = driver.find_elements_by_class_name(comment_class_when_scrolling)
            pre_scroll_num_of_comment= len(visible_comments)         
            last_visible_comment = visible_comments[-1]
            actions = ActionChains(driver)
            actions.move_to_element(last_visible_comment)
            actions.perform()
            time.sleep(1)
            last_visible_comment.location_once_scrolled_into_view
            visible_comments = driver.find_elements_by_class_name(comment_class_when_scrolling)
            num_current_comment_found = len(visible_comments)
html=driver.page_source
driver.close()
js_comment=BeautifulSoup(html,features="lxml")

In [25]:
def getRoomcomments(soupage):
    crooms=soupage.findAll('div', {'class': '_1gjypya'})
    result=[]
    for room in crooms:
        result.append(room)
    return result
def getcomments(result):
    comments=[]
    for i in result:
        comments.append(i.find('div', {'class': '_1y6fhhr'}).text)
    return comments

## extraction
dcomments=[]
dcomments.append(getcomments(getRoomcomments(js_comment)))
dcomments

[['L’appartement est juste magnifique rien à dire . Je vous le recommande fort. Pia est super réactif. La vue est juste magnifique à couper le souffle l’appartement est propres. Très bien situé',
  'Très beau séjour ! Nous avons adoré la vue et la réactivité de Pia et Jean Paul',
  'Le logement est conforme avec une vue à couper le souffle visible dès l’entrée dans les lieux ! Très bien situé dans le quartier d’affaire de La Défense. Les hôtes sont disponibles et très agréables',
  'Très beau séjour ! A recommander ',
  'Tres belle vue',
  'La vue à sauvé le séjour, sinon pas très emballer des lieux. Manque de propreté, télécommande introuvable, tous les tableaux décrochés. \nBien dommage pour une journée d’anniversaire.',
  'La vue est vraiment superbe. \nHôte vraiment sympa et serviable\nUn lieu que je recommande.',
  'Superbe appartement avec une vue exceptionnelle !! \nPia est super réactive et adorable ! \nJe recommande grandement.\nA réserver sans hésiter !',
  'La vue est époust